### TUTORIAL

In [3]:
import pandas as pd
import numpy as np
import csv
from sklearn.utils import Bunch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

from gensim.models import Word2Vec
import nltk
from gensim.models import KeyedVectors

from nltk.cluster import KMeansClusterer
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
import numpy as np 

from sklearn import cluster
from sklearn import metrics

from tensorflow.keras.preprocessing.text import one_hot
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [4]:
    categories = ["not_sexist", "sexist"]
    #               2161           989

In [5]:
data = pd.read_csv("my_csv_clean.csv",sep = ',')
data.columns = ['tweet', 'class']

X = data['tweet']
y = data['class']

X_train, X_test, y_train , y_test = train_test_split(X , y ,test_size=0.3)

### Tokenizing text with scikit-learn

In [6]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(2203, 9650)

In [7]:
count_vect.vocabulary_.get(u'femme')

3606

### From occurrences to frequencies

In [8]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2203, 9650)

### Training a classifier

In [9]:
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [10]:
docs_new = ["Les hommes c'est tous les mêmes", 'Il est étudiant.'] #marche pas du tout
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, categories[category]))

"Les hommes c'est tous les mêmes" => not_sexist
'Il est étudiant.' => not_sexist


### Building a pipeline

In [11]:
text_clf = Pipeline([
...     ('vect', CountVectorizer()),
...     ('tfidf', TfidfTransformer()),
...     ('clf', MultinomialNB()),
... ])
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

### Evaluation of the performance on the test set

In [12]:
docs_test = X_test
predicted = text_clf.predict(docs_test)
np.mean(predicted == y_test)

0.6973544973544974

### SVM

In [13]:
text_clf = Pipeline([
...     ('vect', CountVectorizer()),
...     ('tfidf', TfidfTransformer()),
...     ('clf', SGDClassifier(loss='hinge', penalty='l2',
...                           alpha=1e-3, random_state=42,
...                           max_iter=5, tol=None)),
... ])
text_clf.fit(X_train, y_train)
predicted = text_clf.predict(docs_test)
np.mean(predicted == y_test)

0.7301587301587301

In [14]:
print(metrics.classification_report(y_test, predicted,
...     target_names=categories))

              precision    recall  f1-score   support

  not_sexist       0.74      0.93      0.82       645
      sexist       0.66      0.30      0.42       300

    accuracy                           0.73       945
   macro avg       0.70      0.62      0.62       945
weighted avg       0.72      0.73      0.69       945



In [15]:
metrics.confusion_matrix(y_test, predicted)

array([[599,  46],
       [209,  91]], dtype=int64)

### Parameter tuning using grid search

In [16]:
parameters = {
...     'vect__ngram_range': [(1, 1), (1, 2)],
...     'tfidf__use_idf': (True, False),
...     'clf__alpha': (1e-2, 1e-3),
... }

In [17]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [18]:
gs_clf = gs_clf.fit(X_train[:100], y_train[:100])
#ameliorer vecteur et algo et representation
#-> plus de poids sur les mots sexistes

In [19]:
example_1 = 'La femme'
example_2 = "L'homme"
example_3 = "La femme doit être dans la cuisine"

print(example_1 + "  => " + str(categories[gs_clf.predict([example_1])[0]]))
print(example_2 + "  => " + str(categories[gs_clf.predict([example_2])[0]]))
print(example_3 + "  => " + str(categories[gs_clf.predict([example_3])[0]]))

La femme  => sexist
L'homme  => sexist
La femme doit être dans la cuisine  => sexist


In [20]:
gs_clf.best_score_

0.82

In [21]:
for param_name in sorted(parameters.keys()):
     print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: False
vect__ngram_range: (1, 1)


In [88]:
predicted = gs_clf.predict(X_test)
np.mean(predicted == y_test)
print("Accuracy : ", np.mean(predicted == y_test))
metrics.confusion_matrix(y_test, predicted)

Accuracy :  0.7111111111111111


array([[587,  68],
       [205,  85]], dtype=int64)

### Oversampling

In [22]:
# instantiating the random over sampler 
ros = RandomOverSampler()
# resampling X, y
X_ros, y_ros = ros.fit_resample(np.array(X_train).reshape(-1,1), y_train)# new class distribution 
print(Counter(y_ros))

Counter({0: 1514, 1: 1514})


In [23]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_ros.ravel())
X_train_counts.shape

(3028, 9650)

In [24]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(3028, 9650)

In [25]:
clf = MultinomialNB().fit(X_train_tfidf, y_ros)
docs_new = ["y'a que les femmes qui pleurent", "C'est un homme."] 
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, categories[category]))
        
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
text_clf.fit(X_ros.ravel(), y_ros)


# resampling X, y
X_ros_test, y_ros_test = ros.fit_resample(np.array(X_test).reshape(-1,1), y_test)# new class distribution 
docs_test = X_ros_test.ravel()
predicted = text_clf.predict(docs_test)
np.mean(predicted == y_ros_test)
print("Accuracy : ", np.mean(predicted == y_ros_test))
metrics.confusion_matrix(y_ros_test, predicted)

"y'a que les femmes qui pleurent" => sexist
"C'est un homme." => sexist
Accuracy :  0.7325581395348837


array([[430, 215],
       [130, 515]], dtype=int64)

### Undersampling

In [26]:
# instantiating the random over sampler 
ros = RandomUnderSampler()
# resampling X, y
X_ros, y_ros = ros.fit_resample(np.array(X_train).reshape(-1,1), y_train)# new class distribution 
print(Counter(y_ros))

Counter({0: 689, 1: 689})


In [27]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_ros.ravel())
X_train_counts.shape

(1378, 7124)

In [28]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(1378, 7124)

In [29]:
clf = MultinomialNB().fit(X_train_tfidf, y_ros)
docs_new = ["y'a que les femmes qui pleurent", "C'est un homme."] 
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, categories[category]))
        
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
text_clf.fit(X_ros.ravel(), y_ros)


# resampling X, y
X_ros_test, y_ros_test = ros.fit_resample(np.array(X_test).reshape(-1,1), y_test)# new class distribution 
docs_test = X_ros_test.ravel()
predicted = text_clf.predict(docs_test)
print("Accuracy : ", np.mean(predicted == y_ros_test))
metrics.confusion_matrix(y_ros_test, predicted)


"y'a que les femmes qui pleurent" => sexist
"C'est un homme." => sexist
Accuracy :  0.7283333333333334


array([[183, 117],
       [ 46, 254]], dtype=int64)

### Oversampling and Undersampling

In [30]:
over = RandomOverSampler(sampling_strategy=0.5)
under = RandomUnderSampler(sampling_strategy=0.8)

In [31]:
X_over, y_over = over.fit_resample(np.array(X_train).reshape(-1,1), y_train)
print(f"Oversampled: {Counter(y_over)}")

Oversampled: Counter({0: 1514, 1: 757})


In [32]:
# now to comine under sampling 
X_ros, y_ros = under.fit_resample(X_over, y_over)
print(f"Combined Random Sampling: {Counter(y_ros)}")

Combined Random Sampling: Counter({0: 946, 1: 757})


In [33]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_ros.ravel())
X_train_counts.shape

(1703, 8047)

In [34]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(1703, 8047)

In [35]:
clf = MultinomialNB().fit(X_train_tfidf, y_ros)
docs_new = ["y'a que les femmes qui pleurent", "C'est un homme."]
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, categories[category]))
        
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
text_clf.fit(X_ros.ravel(), y_ros)


# resampling X, y
X_ros_test, y_ros_test = ros.fit_resample(np.array(X_test).reshape(-1,1), y_test)# new class distribution 
docs_test = X_ros_test.ravel()
predicted = text_clf.predict(docs_test)
print("Accuracy : ", np.mean(predicted == y_ros_test))
metrics.confusion_matrix(y_ros_test, predicted)


"y'a que les femmes qui pleurent" => not_sexist
"C'est un homme." => sexist
Accuracy :  0.7066666666666667


array([[229,  71],
       [105, 195]], dtype=int64)

### Naive Bayes classifier

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

data = pd.read_csv("my_csv_clean.csv",sep = ',') #we got that csv after running the Text Preprocessing file
data.columns = ['tweet', 'class']
    

X = data['tweet']
y = data['class'] 


categories = ["NotSexist", "Sexist"]

X_0 = []
X_1 = []
for i in range(len(data)):
    if data['class'][i]==0:
        X_0.append(data['tweet'][i])
        
for i in range(len(data)):
    if data['class'][i]==1:
        X_1.append(data['tweet'][i])



vec_0 = CountVectorizer()
X_c0 = vec_0.fit_transform(X_0)
tdm_0 = pd.DataFrame(X_c0.toarray(), columns=vec_0.get_feature_names())


vec_1 = CountVectorizer()
X_c1 = vec_1.fit_transform(X_1)
tdm_1 = pd.DataFrame(X_c1.toarray(), columns=vec_1.get_feature_names())

tdm_1


C:\Users\evara\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,02,04,06,10,100,1000,102,12,120,12000,...,yqb,yves,zaimerais,zamzonite,zenash,zeniss,zetais,zlatan,zohrabitan,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
word_list_0 = vec_0.get_feature_names();    
count_list_0 = X_c0.toarray().sum(axis=0) 
freq_0 = dict(zip(word_list_0,count_list_0))
freq_0

C:\Users\evara\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'000': 4,
 '10': 7,
 '100': 10,
 '1000': 1,
 '102': 2,
 '10h': 2,
 '10h00': 1,
 '10h30': 1,
 '11': 4,
 '1100': 1,
 '11000': 1,
 '117': 1,
 '12': 8,
 '1200': 1,
 '12000': 1,
 '120battementsparminute': 1,
 '1215': 1,
 '12h': 1,
 '12joursdaction': 1,
 '13': 7,
 '13h50': 1,
 '14': 9,
 '14eme': 1,
 '14h': 1,
 '14h00': 1,
 '14h15': 2,
 '15': 11,
 '157garlamd': 1,
 '15min': 1,
 '15x': 1,
 '16': 4,
 '169': 1,
 '16days': 2,
 '16h': 3,
 '16jours': 3,
 '17': 7,
 '1700': 1,
 '1703': 1,
 '1730': 1,
 '18': 9,
 '180': 1,
 '18006675005': 1,
 '18006686868': 1,
 '1825': 1,
 '18711955': 1,
 '18h': 2,
 '18h00': 1,
 '18h30': 1,
 '18h55': 1,
 '19': 6,
 '1900': 1,
 '1904': 1,
 '1905': 1,
 '1920': 2,
 '1927': 2,
 '1929': 2,
 '195': 2,
 '1954': 1,
 '1962': 1,
 '1965': 1,
 '1970': 1,
 '1979': 1,
 '1986': 1,
 '1987': 1,
 '1992': 1,
 '1999': 1,
 '19e': 1,
 '19h': 1,
 '19h30': 1,
 '19hruthelkrief': 2,
 '1a0maiscpasperdu': 1,
 '1dexmab': 1,
 '1dexmag': 1,
 '1e': 1,
 '1er': 10,
 '1ere': 17,
 '1iran': 1,
 '1les': 1,

In [38]:
word_list_1 = vec_1.get_feature_names();    
count_list_1 = X_c1.toarray().sum(axis=0) 
freq_1 = dict(zip(word_list_1,count_list_1))
freq_1

{'02': 3,
 '04': 3,
 '06': 1,
 '10': 6,
 '100': 2,
 '1000': 1,
 '102': 1,
 '12': 5,
 '120': 2,
 '12000': 1,
 '12f': 1,
 '13': 4,
 '14': 5,
 '14amp15': 1,
 '14ans': 1,
 '15': 2,
 '150eur': 1,
 '15ans': 1,
 '16': 2,
 '16ans': 1,
 '16days': 2,
 '16jours': 3,
 '17': 1,
 '1964': 1,
 '1990': 1,
 '1992': 2,
 '1997': 2,
 '1998': 1,
 '1999': 1,
 '1cm': 1,
 '1dexmag': 1,
 '1er': 1,
 '1ere': 3,
 '20': 7,
 '2004': 1,
 '2005': 1,
 '2006': 1,
 '2011': 1,
 '20122017': 1,
 '2014': 1,
 '2015': 2,
 '2016': 2,
 '2017': 9,
 '2018': 15,
 '20ans': 1,
 '20hfrance2': 1,
 '20minutes': 2,
 '21': 2,
 '21eme': 2,
 '22': 4,
 '23': 1,
 '24': 1,
 '2499': 1,
 '24hpujadas': 1,
 '25': 2,
 '25janvier': 2,
 '26': 1,
 '26caroll': 1,
 '27': 2,
 '28': 1,
 '2choses': 1,
 '2h': 1,
 '30': 4,
 '31': 1,
 '32': 1,
 '36': 1,
 '3600x': 1,
 '364': 1,
 '3784e': 1,
 '38': 2,
 '3eme': 1,
 '3min': 1,
 '40': 1,
 '40t': 1,
 '44': 1,
 '44t': 1,
 '4h': 1,
 '4meur': 1,
 '50': 5,
 '500': 1,
 '5000': 1,
 '50100500': 1,
 '50euros': 1,
 '50nuanc

In [39]:
prob_0 = []
for word,count in zip(word_list_0,count_list_0):
    prob_0.append(count/len(word_list_0))
dict(zip(word_list_0,prob_0))


{'000': 0.00043677658877484165,
 '10': 0.0007643590303559729,
 '100': 0.0010919414719371041,
 '1000': 0.00010919414719371041,
 '102': 0.00021838829438742082,
 '10h': 0.00021838829438742082,
 '10h00': 0.00010919414719371041,
 '10h30': 0.00010919414719371041,
 '11': 0.00043677658877484165,
 '1100': 0.00010919414719371041,
 '11000': 0.00010919414719371041,
 '117': 0.00010919414719371041,
 '12': 0.0008735531775496833,
 '1200': 0.00010919414719371041,
 '12000': 0.00010919414719371041,
 '120battementsparminute': 0.00010919414719371041,
 '1215': 0.00010919414719371041,
 '12h': 0.00010919414719371041,
 '12joursdaction': 0.00010919414719371041,
 '13': 0.0007643590303559729,
 '13h50': 0.00010919414719371041,
 '14': 0.0009827473247433938,
 '14eme': 0.00010919414719371041,
 '14h': 0.00010919414719371041,
 '14h00': 0.00010919414719371041,
 '14h15': 0.00021838829438742082,
 '15': 0.0012011356191308146,
 '157garlamd': 0.00010919414719371041,
 '15min': 0.00010919414719371041,
 '15x': 0.000109194147193

In [40]:
prob_1 = []
for word,count in zip(word_list_1,count_list_1):
    prob_1.append(count/len(word_list_1))
dict(zip(word_list_1,prob_1))


{'02': 0.0005337128624799857,
 '04': 0.0005337128624799857,
 '06': 0.0001779042874933286,
 '10': 0.0010674257249599715,
 '100': 0.0003558085749866572,
 '1000': 0.0001779042874933286,
 '102': 0.0001779042874933286,
 '12': 0.000889521437466643,
 '120': 0.0003558085749866572,
 '12000': 0.0001779042874933286,
 '12f': 0.0001779042874933286,
 '13': 0.0007116171499733144,
 '14': 0.000889521437466643,
 '14amp15': 0.0001779042874933286,
 '14ans': 0.0001779042874933286,
 '15': 0.0003558085749866572,
 '150eur': 0.0001779042874933286,
 '15ans': 0.0001779042874933286,
 '16': 0.0003558085749866572,
 '16ans': 0.0001779042874933286,
 '16days': 0.0003558085749866572,
 '16jours': 0.0005337128624799857,
 '17': 0.0001779042874933286,
 '1964': 0.0001779042874933286,
 '1990': 0.0001779042874933286,
 '1992': 0.0003558085749866572,
 '1997': 0.0003558085749866572,
 '1998': 0.0001779042874933286,
 '1999': 0.0001779042874933286,
 '1cm': 0.0001779042874933286,
 '1dexmag': 0.0001779042874933286,
 '1er': 0.00017790

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

X_train, X_test, y_train , y_test = train_test_split(X , y ,test_size=0.3)
X_train_0 = []
X_train_1 = []
print(X_train)
print(X_train.index)

index =  X_train.index
for i in index:
    if (y_train[i]==0):
        X_train_0.append(X_train[i])
    else:
        X_train_1.append(X_train[i])       

318     avant de choisir prenom un garcon pensez femme...
619                    angelslovess as balancetonporc mdr
179     metoo balancetonporc  avenement du matriarcat ...
1692    les francais connaissent difficultes de foncti...
591     paris  station de metro europe sur ligne 3 va ...
                              ...                        
3111    instances intellectuelles ces territoires excl...
2777    il bon de rappeler parfois choses avancent gra...
2740    est bien comme couverture de foutre raciste ho...
604     csaudiovisuel condamne france2tv pour reportag...
1327    equipedefrance orangevelodrome c8tv allez mesd...
Name: tweet, Length: 2203, dtype: object
Int64Index([ 318,  619,  179, 1692,  591, 1402, 2011, 2975, 2442, 1387,
            ...
            2727, 1498,  729, 2965,   34, 3111, 2777, 2740,  604, 1327],
           dtype='int64', length=2203)


In [42]:
vec_0 = CountVectorizer()
X_vec_0 = vec_0.fit_transform(X_train_0)

total_features0 = len(vec_0.get_feature_names())
total_features0


7279

In [43]:
vec_1 = CountVectorizer()
X_vec_1 = vec_1.fit_transform(X_train_1)

total_features1 = len(vec_1.get_feature_names())
total_features1

proba0 = total_features0 / (total_features0+total_features1)
proba1 = total_features1 / (total_features0+total_features1)

print(proba0)
print(proba1)

0.6205456095481671
0.37945439045183293


In [44]:
total_cnts_features_0 = count_list_0.sum(axis=0)
total_cnts_features_1 = count_list_1.sum(axis=0)

In [45]:
from nltk.tokenize import word_tokenize
new_sentence = 'what is the price of the book'
new_word_list = word_tokenize(new_sentence)

In [46]:
def proba_sentence_class0(sentence):
    new_word_list = word_tokenize(sentence)
    prob_s_with_ls = []
    for word in new_word_list:
        if word in freq_0.keys():
            count = freq_0[word]
        else:
            count = 0
        prob_s_with_ls.append((count + 1)/(total_cnts_features_0 + total_features0))
    d = dict(zip(new_word_list,prob_s_with_ls))
    res = 1
    for word in sentence.split():
        res = res * d[word]

    res = res * proba0
    return res


In [47]:
def proba_sentence_class1(sentence):
    new_word_list = word_tokenize(sentence)
    prob_s_with_ls = []
    for word in new_word_list:
        if word in freq_1.keys():
            count = freq_1[word]
        else:
            count = 0
        prob_s_with_ls.append((count + 1)/(total_cnts_features_1 + total_features1))
    d = dict(zip(new_word_list,prob_s_with_ls))
    res = 1
    for word in sentence.split():
        res = res * d[word]

    res = res * proba1
    return res

print(proba_sentence_class1("what is the price of the book"))

2.05406836886256e-31


In [48]:
predictions = []
for i in X_test:
    print(i)
    proba_class0 = proba_sentence_class0(i)
    proba_class1 = proba_sentence_class1(i)
    if proba_class0 > proba_class1:
        predictions.append(0)
    else:
        predictions.append(1)
        
print(predictions)

angela merkel nouveau  cap sur afrique
trudeau plaide faveur du mouvement moiaussi
connasse cette grosse vache faut quelle redescende lrdmv
prendre 1 femme enceinte otage ne donne pas cher de vos vies crapauds  pour tir 1 cartouche pleine tete approvisionnez
jaime video youtube  taharrush gamea  ou viol autorise adresse
christiane taubira   chaque fois quon restreint libertes on na pas su retablir  pjlterrorisme
1 femme voilee voulait laisser place ds ltram 2 vieilles yen 1 qui lenvoie chier pcq elle voilee sale pute reste debout alors
heureusement on frequente pas avec cette fille ironie lrt
argent ambition pouvoir trois termes qui prennent tonalite negative quand on associe aux femmes il temps de deconstruire ces modeles pour briser plafond de verre via lesechos sexisme
quand ministre mis cause pour viol il ne peut pas rester gouvernement
tcabarrus europe1 moins elle sait lire on parle salaires footballeurs sexisme
si afrique tres presente dans nouvel accord de coalition gouvernement

par contre rien sur balancetonbabybel  tout sur balancetonporc  avouez quil y quand meme probleme dans ce pays vous pensez quoi 
philousports matblaize putain vieux tir de femme enceinte ligue1 webgirondins
actrices ont parfaitement explique pourquoi elles portaient toutes du noir aux goldenglobes sexisme hollywood
comment ca sur insta y gens qui postent par theme mode photos bleues samedi prochain pis apres est rose
harcelement sexuel sur lieu du travail  dans temps on parlait plus de droit de cuissage de promotion canape etc langues delient aux usa metoo balancetonporc
ce matin collectif ensemblecontrelesexisme reunit pour finaliser programme de 1re journee nationale contre sexisme 25 janv 8h30  18h chez telecomptech inscriptions franceegalite tftc nomorecliches
riri happy elle fait performance de fou vous voyez grosseur de seins bide si est pas du sexisme ca  jamais ca arrive aux mecs bordel
moiaussi dis  etmaintenant 
balancetonporc  liberation de parole sous caution mediatique 
se

moi base pensais balancetonporc etait nouvelle expression qui allait remplacer ca envoie du pate 
on connait tous aurore berge est moche qui traine avec belles qui compense avec de indecence 1cm de maquillage pour faire illusion 4h du ma heure laquelle lui faire bise revient laver visage avec beignet plein mois aout
brownskinstrauw non mais est balancetonporc 
melaniarosnthal aliaslautre 54lilly franceculture bof jai lu heureusement ce nest 5mn mais titre me choque profondement amp de part de fr culture ca ne passe pas sexiste
ok petit thread balancetonporc  parce suis vraiment choquee quil faut jen parle
petite fille qui joue faire sandwich avec de boue comme est mignon thewalkingdead ironie
balancetonporc lionel sanchez sanchezlionel de radio francebleu viole enfants de 12 ans rouen college jean baptiste de salle alors quil employe comme surveillant balancetonpedophile il devrait etre prison lieu de ca il libre
jour ou enfin sexe pu ressembler fantasme cheap faiseurs de films pornosi

sur discours oprah aux goldenglobe hier   etait tres puissant etait facon de dire aux hommes est temps ca change   mlamont6 moiaussi metoo
lettreaudio gofundme si vous nous dites date de audience on y sera  balancetonporc
poursuivie diffamation creatrice de balancetonporc lance collecte pour payer defense
il faut enormement de courage en parler devant milliers voire millions de personnes on va etre juge voire moque est raison pour laquelle soutiens mouvements balancetonporc metoo peu importe ce autres pensent
photo de theresa may bruxelles suscite risee internautes
decider mise pied de tex resultat on glose sur legitimite de sanction lieu de indigner de blague sexiste inadmissible
affaire buttet qui profite crime  buttet harcelement moiaussi metoo balancetonporc 1dexmab politique sexisme
benzema =tir de femme enceintefranig
pote moi est fait claque fesses soiree balancetonporc
brunolemaire ademontchalin fait donc cette femme enfoulardee derriere vous  egalite femmeshommes dites vousvou

tiramitsu tpkssb balancetonporc  litteralement
angela merkel garde couronne de femme plus puissante de planete exception de mme merkel au 1er r
oliviergiroud etais serieux avec tir de femme enceinte 
alors donc faut dire mesdames si vous avez malheur de bosser avec votre mari vous ne serez jamais collaboratrice parce vous comprenez vous couchez aussi avec est lui qui aura tout credit bien sur sexisme
pauvre type qui fait du buzz avec propres dechets qui boit de biere dans train  part alcoolique bouffi sans talent qui utilise balancetonporc sauce g mal aux couilles
moodydisorder ailleurs transphobie sexisme sont pas souvent relies comme ca  nan 
sacrilege pour uns benediction pour autres  quand minijupe debarque paris
magnifique lettre opinion signee par survivante sur apres moiaussi  en octobre dernier ai dit
comment aider collegue victime de harcelement sexuel qui vous demande de ne rien dire personne  balancetonporc
allez bleues   train de regarder match de handball
je ne pense pas q

...

tout monde pense quil ne agit hollywood ce nest pas juste hollywood est monde  rose mcgowan actrice activiste victime metoo balancetonporc
8mars aujourhui on passera aussi du cote de assembleenat ecouter interventions dont payetashnek ramsellem sur liberation de parole reflexions necessaires alors elues personnalites publiques ont apparente balancetonporc de delation
prochain cafeministe avec sandrine rousseau sur sexisme politique dans militantes venez nombreuses nombreux lieudit 16h sandrinerousseau cafe lelieudit 20eme politique sexisme eelv militantisme feminisme egalitefemmehomm
present groupe de travail lutte contre sexisme harcelement toutes formes de discriminations avec lafhf cpuniversite aphp isnitwit karinelacour kevinneuville doyenne dentaire dir aff med dg ch drh chu dg fp estfhf tous unis pour egalite 
bjr canardenchaine ce frederic haziza suspendu de lcp pr agressions sexuelles travaille encore pr vs  balancetonporc
emmanuel macron angela merkel mettent cap sur reforme e

bonne chance nos bleues pour leur match contre italie ce soir orangevelodrome  fraita allezlesbleues
prenez soin de vous avec ce mauvais temps balancetonporc feudechemineecabworcester
on demande survivantes message elles auraient aime entendre durant temps fetes voici ce quelles nous ont dit  moiaussi onvouscroit
on ne fait pas une salope femme foyer
moiaussi  centaines de personnes manifestent toronto
dernier blog reviens maniere sur moiaussi du processus jai fais pour denoncer
balancetonporc grudulepoulpe suis pas petite salope
22existetil moindre etude comparative prouvant notre epoque plus raciste  ou plus sexiste est meme tambouille  que par exemple annee 1992  estce ce ne serait pas plutot contraire 
jose porter bottes talons pajol paris18  jai eu droit 3 coquine salope 3min rien ne vous choque mairie18paris  on laisse comme ca tout va bien  on ne occupe tjs pas de place femmes dans quartier  balancetonporc marleneschiappa
presentation reformes gouvernementales commerce exterieur

theresa may tente de sortir brexit de impasse
queenelisar alors si es pas accord ca veut dire non consentante donc est viol non pas relation saine  du haut de 14 ans ne sais rien vie
travail domicile google home max  enceinte connectee rend musique explosive
ce serait perte du privilege du corps enseignant etre juge exclusivement par pairs qui motiverait executif du sgpum dans combat contre modification de charte de universite de montreal moiaussi
angela merkel pour 5 e fois femme plus puissante du monde
ankara juge inacceptables propos tenus par angela merkel concernant operation afrine
premiere journeenationalecontrelesexisme 25 janvier organisee par collectif ensemblecontrelesexisme  inscrivezvous  egalite egalitefh
reine neiges star wars marvel bientot trois nouveaux parcs disneylandparis ecoutez robertiger pdg de walt disney company recu elysee ce matin nous dire pourquoi il continue investir france choosefrance
 vpecresse doit cesser greve agir pour obtenir remboursement du navig

meuf saoule tout monde avec grossophobie elle fait du sexisme 
quon ne vienne pas me dire jai esprit mal place hein  ca voit comme bite milieu de figure ensemblecontrelesexisme
video  furieuse christiane taubira quitte interview de complement enquete
jour ou deferlante actuelle de denonciations sera suffisamment forte pour ces victimes laissees ellesmemes y joignent aussi sera tres grand jour pour femmes ecrit joseelegault moiaussi
padmavatifabricatedmuslimshistory nous sommes pas contre stars du ce film mais nous sommes contre slb car il presente roi musulmankhilji comme homme mal  gay berber c catastrophepls respect history padmaavat deepikapadukone bdmftysyllmslmyn
gnarf on retrouve meme coalition de mecs qui defendent uns autres comme moment de affaire dsk yen encore qui nont rien dit ils sont devenus muets  metoo balancetonporc
2006 javais 20 ans dans metro toulousain colle derriere moi tu excite me suis hotel sortie balancetonporc
aurore berge exprime sur harcelement 
inconduite 

petit thread sur anais bourdet de payetashnek feminisme geometrie variable
mormach moi mange nature elle na pas temps avoir peur sauf moment du viol quand ouvre 
collectif ensemble contre sexisme organise 1ere journee nationale contre sexisme avec sequence dediee entreprise cet evenement gratuit tiendra ecole telecomparistech paris 13 de 8h30 18h 25 janvier cc talentuelles
apres titre coup de poing de chillaofficiel balancetonporc freestyle poignant metoo de vinswaza enfonce clou
de balancetonporc tariq ramadan new yorker analyse notre debat public est pas beau voir
joli moment emotion avec coup envoi fictif de jolie retraitee louisanecib legende du foot feminin pour match fraita stade velodrome allez bleues 
facebook censure teaser de notre webserie car on y voit seins si on arretait de systematiquement sexualiser corps de femme  il ny absolument rien obscene montrer anatomie humaine  sexismeordinaire sexisme sexualisation femmes discrimination
ce photographe inverse roles pour denonc

In [49]:
total_labels = len(y_test)
acc = 0
for i in range(len(y_test)):
    if(y_test[y_test.index[i]]==predictions[i]):
        acc = acc + 1
        
acc = acc / total_labels
print("Accuracy :" , acc)
        
metrics.confusion_matrix(y_test, predictions)

Accuracy : 0.8328042328042328


array([[493, 155],
       [  3, 294]], dtype=int64)

### NAIVES BAYES + OVERSAMPLING

In [50]:
data = pd.read_csv("my_csv_clean.csv",sep = ',') #we got that csv after running the Text Preprocessing file
data.columns = ['tweet', 'class']
    

X = data['tweet']
y = data['class'] 


categories = ["NotSexist", "Sexist"]

X_0 = []
X_1 = []
for i in range(len(data)):
    if data['class'][i]==0:
        X_0.append(data['tweet'][i])
        
for i in range(len(data)):
    if data['class'][i]==1:
        X_1.append(data['tweet'][i])

vec_0 = CountVectorizer()
X_c0 = vec_0.fit_transform(X_0)
tdm_0 = pd.DataFrame(X_c0.toarray(), columns=vec_0.get_feature_names())


vec_1 = CountVectorizer()
X_c1 = vec_1.fit_transform(X_1)
tdm_1 = pd.DataFrame(X_c1.toarray(), columns=vec_1.get_feature_names())

tdm_1

C:\Users\evara\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,02,04,06,10,100,1000,102,12,120,12000,...,yqb,yves,zaimerais,zamzonite,zenash,zeniss,zetais,zlatan,zohrabitan,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
word_list_0 = vec_0.get_feature_names();    
count_list_0 = X_c0.toarray().sum(axis=0) 
freq_0 = dict(zip(word_list_0,count_list_0))
freq_0

C:\Users\evara\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'000': 4,
 '10': 7,
 '100': 10,
 '1000': 1,
 '102': 2,
 '10h': 2,
 '10h00': 1,
 '10h30': 1,
 '11': 4,
 '1100': 1,
 '11000': 1,
 '117': 1,
 '12': 8,
 '1200': 1,
 '12000': 1,
 '120battementsparminute': 1,
 '1215': 1,
 '12h': 1,
 '12joursdaction': 1,
 '13': 7,
 '13h50': 1,
 '14': 9,
 '14eme': 1,
 '14h': 1,
 '14h00': 1,
 '14h15': 2,
 '15': 11,
 '157garlamd': 1,
 '15min': 1,
 '15x': 1,
 '16': 4,
 '169': 1,
 '16days': 2,
 '16h': 3,
 '16jours': 3,
 '17': 7,
 '1700': 1,
 '1703': 1,
 '1730': 1,
 '18': 9,
 '180': 1,
 '18006675005': 1,
 '18006686868': 1,
 '1825': 1,
 '18711955': 1,
 '18h': 2,
 '18h00': 1,
 '18h30': 1,
 '18h55': 1,
 '19': 6,
 '1900': 1,
 '1904': 1,
 '1905': 1,
 '1920': 2,
 '1927': 2,
 '1929': 2,
 '195': 2,
 '1954': 1,
 '1962': 1,
 '1965': 1,
 '1970': 1,
 '1979': 1,
 '1986': 1,
 '1987': 1,
 '1992': 1,
 '1999': 1,
 '19e': 1,
 '19h': 1,
 '19h30': 1,
 '19hruthelkrief': 2,
 '1a0maiscpasperdu': 1,
 '1dexmab': 1,
 '1dexmag': 1,
 '1e': 1,
 '1er': 10,
 '1ere': 17,
 '1iran': 1,
 '1les': 1,

In [52]:
word_list_1 = vec_1.get_feature_names();    
count_list_1 = X_c1.toarray().sum(axis=0) 
freq_1 = dict(zip(word_list_1,count_list_1))
freq_1

{'02': 3,
 '04': 3,
 '06': 1,
 '10': 6,
 '100': 2,
 '1000': 1,
 '102': 1,
 '12': 5,
 '120': 2,
 '12000': 1,
 '12f': 1,
 '13': 4,
 '14': 5,
 '14amp15': 1,
 '14ans': 1,
 '15': 2,
 '150eur': 1,
 '15ans': 1,
 '16': 2,
 '16ans': 1,
 '16days': 2,
 '16jours': 3,
 '17': 1,
 '1964': 1,
 '1990': 1,
 '1992': 2,
 '1997': 2,
 '1998': 1,
 '1999': 1,
 '1cm': 1,
 '1dexmag': 1,
 '1er': 1,
 '1ere': 3,
 '20': 7,
 '2004': 1,
 '2005': 1,
 '2006': 1,
 '2011': 1,
 '20122017': 1,
 '2014': 1,
 '2015': 2,
 '2016': 2,
 '2017': 9,
 '2018': 15,
 '20ans': 1,
 '20hfrance2': 1,
 '20minutes': 2,
 '21': 2,
 '21eme': 2,
 '22': 4,
 '23': 1,
 '24': 1,
 '2499': 1,
 '24hpujadas': 1,
 '25': 2,
 '25janvier': 2,
 '26': 1,
 '26caroll': 1,
 '27': 2,
 '28': 1,
 '2choses': 1,
 '2h': 1,
 '30': 4,
 '31': 1,
 '32': 1,
 '36': 1,
 '3600x': 1,
 '364': 1,
 '3784e': 1,
 '38': 2,
 '3eme': 1,
 '3min': 1,
 '40': 1,
 '40t': 1,
 '44': 1,
 '44t': 1,
 '4h': 1,
 '4meur': 1,
 '50': 5,
 '500': 1,
 '5000': 1,
 '50100500': 1,
 '50euros': 1,
 '50nuanc

In [77]:
prob_0 = []
for word,count in zip(word_list_0,count_list_0):
    prob_0.append(count/len(word_list_0))
#dict(zip(word_list_0,prob_0))


In [78]:
prob_1 = []
for word,count in zip(word_list_1,count_list_1):
    prob_1.append(count/len(word_list_1))
#dict(zip(word_list_1,prob_1))


In [55]:
from sklearn.feature_extraction.text import CountVectorizer

X_train, X_test, y_train , y_test = train_test_split(X , y ,test_size=0.3)

# instantiating the random over sampler 
ros = RandomOverSampler()
# resampling X, y
X_ros_train, y_ros_train = ros.fit_resample(np.array(X_train).reshape(-1,1), y_train)# new class distribution 
X_ros_test, y_ros_test = ros.fit_resample(np.array(X_test).reshape(-1,1), y_test)# new class distribution 

X_train_0 = []
X_train_1 = []

for i in range(len(X_ros_train)):
    if (y_ros_train[i]==0):
        X_train_0.append(X_ros_train[i].tolist()[0])
    else:
        X_train_1.append(X_ros_train[i].tolist()[0])

In [56]:
vec_0 = CountVectorizer()
X_vec_0 = vec_0.fit_transform(X_train_0)

total_features0 = len(vec_0.get_feature_names())
total_features0


C:\Users\evara\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


7216

In [57]:
vec_1 = CountVectorizer()
X_vec_1 = vec_1.fit_transform(X_train_1)

total_features1 = len(vec_1.get_feature_names())
total_features1

proba0 = total_features0 / (total_features0+total_features1)
proba1 = total_features1 / (total_features0+total_features1)

print(proba0)
print(proba1)

0.619452313503305
0.380547686496695


In [58]:
total_cnts_features_0 = count_list_0.sum(axis=0)
total_cnts_features_1 = count_list_1.sum(axis=0)

In [59]:
from nltk.tokenize import word_tokenize
new_sentence = 'what is the price of the book'
new_word_list = word_tokenize(new_sentence)

In [80]:
def proba_sentence_class0(sentence):
    new_word_list = word_tokenize(sentence)
    prob_s_with_ls = []
    for word in new_word_list:
        if word in freq_0.keys():
            count = freq_0[word]
        else:
            count = 0
        prob_s_with_ls.append((count + 1)/(total_cnts_features_0 + total_features0))
    d = dict(zip(new_word_list,prob_s_with_ls))
    res = 1
    for word in sentence.split():
        res = res * d[word]

    res = res * proba0
    return res
print(proba_sentence_class0("Les femmes ne savent pas conduire"))

6.6358704890318526e-21


In [81]:
def proba_sentence_class1(sentence):
    new_word_list = word_tokenize(sentence)
    prob_s_with_ls = []
    for word in new_word_list:
        if word in freq_1.keys():
            count = freq_1[word]
        else:
            count = 0
        prob_s_with_ls.append((count + 1)/(total_cnts_features_1 + total_features1))
    d = dict(zip(new_word_list,prob_s_with_ls))
    res = 1
    for word in sentence.split():
        res = res * d[word]

    res = res * proba1
    return res

print(proba_sentence_class1("Les femmes ne savent pas conduire"))

2.819550276678128e-20


In [82]:
predictions = []
for i in X_ros_test:
    print(i)
    proba_class0 = proba_sentence_class0(i[0])
    proba_class1 = proba_sentence_class1(i[0])
    if proba_class0 > proba_class1:
        predictions.append(0)
    else:
        predictions.append(1)
        
print(predictions)

['agenda 25 janvier 2018 tiendra 1e journee nationale contre sexisme organisee par collectif ensemble contre sexisme journee de rencontres de debats inauguree presence de marleneschiappa infos amp inscriptions egalitefh']
['vpecresse incarner credibilite alors est girouette politique  soutien de sarkozy puis fillon puis juppe puis re fillon maintenant penche serieusement vers macron soeur jumelle de auroreberge ']
['il faut quil partage avec theresa may macron ce nest plus nobel de paix mais nobel de tnt']
['comment greffer neurones con sexiste telle vraie question loser']
['aime comme fou comme soldat comme star de cinema comme loup comme roi comme homme ne suis pas']
['lors de leur rencontre maison blanche aujourhui donald trump qualifie angela merkel de femme extraordinaire']
['est impressionnant comme grand mere defend sexisme mode est normal femmes restent maison est leur role']
['faut vraiment faire confiance personne dans ce monde de salope']
['1 femme voilee voulait laisser pla

['8 mars france2 propose emissions sur femmes toute nuit est bien demain on bosse 8mars feminisme ironie']
['leparisien partiindigenes oceanerosemarie clemautain assolallab mediapartlive edwyplenel ericfassin placebeauvau policenationale vous continuez faire de publicite ces personnes qui font de autodefense leur alpha amp omega bafouent lois de republique avec propos racistes antiroms sexiste amp homphobes  vive scharia est cela ']
['catherine deneuve  je suis femme libre demeurerai  est aux victimes elles elles seules presente excuses via libe']
['johnnyhallyday toujours ete precurseur est pourquoi il ete accuse de viol sur mineure avril 2015 par adeline blondeau soit bien avant balancetonporc  est ca avantgardisme ']
['policerealites veuillez trouver journaliste chanteur de rap vous expliquer est de faute du pompier car est femme']
['etmaintenant ami lecteur lui ai propose de lire ce texte  moiaussi']
[' il passe quelque chose republique islamique iranles gens nont plus peur du regi

['rollandozz scuze fille avec culture musicale incroyable ironie']
['il y avait peu moins de testosterone dans tete de etat etat du monde sera meilleur christiane taubira docteure honoris causa de ulbruxelles bruxelles amp reagit emmanuelmacron trump chtaubira']
['ah donc es feministe toi bah oui ah bah jtaime deja pa enchantee as air etre jeune homme fort sympathique ']
['clemautain vous prefereriez sans aucun doute potiche dans coin de elysee moins sexiste plus feministe plus elegant ']
['merci latribune de porter sujet cote du collectif ensemblecontrelesexisme pour premiere journee contre sexisme dans lequel tres implique tftc cometentreprise numero dedie retrouver kiosque demain latribune aziliz31 jctortora vabrial catladousse']
['je pense si il y avait peu moins de testosterone tete etats du monde monde serait dans meilleur etat christiane taubira']
['benzema =tir de femme enceintefranig']
['wtfloow lolossandre ah merde  balance noms balancetonporc lol jpp ']
['balancetonuber eric

[' joxe il capable du meilleur comme du pire mais est dans pire quil meilleur  metoo balancetonporc']
['alexyrius xliteparadise agatheauproux tpmp balancetonporc alexyrius ce genre de connard qui viole femme parce quelle habillee trop legerement gout']
['native de chalonsenchampagne bleues affrontent italie match amical samedi soir 21 h stade velodrome de marseille football fraita cc unionardennais']
['tres beau texte de simonacrippa  moiaussi viol feminisme']
['4 histoires de feminisme mal place 7eme dans categorie santebien etre  oo ironie jadore']
['mate 2eme episode bayezer de jeanpascalzadi  griezmann manix hampm  retour djihadistes  catherine deneuve']
['merlinclubhouse mualni macron fhollande probleme est macron voulant passer force comme habitude il sousestime capacites de nuisance angela merkel']
['les bleues qualifient finale de cdm de handball saintetienne club historique du championnat train de couler dimanche il y plus gros match du championnat on met quoi mecs  neymar  al

['jackxparker klaire est vrai quoi es fille vas dehors ou il y plein de mecs louches est de faute aussi ironie']
['afrique ou on reconnait quune femme peut occuper poste de responsabilite debrouiller aussi bien quun homme afriquedemesreves']
['yavait balancetonporc perte de vue il y encore qq semaines ajd tous toutes ont mate missfrance2018 mdr feminisme 20 il faut finir avec ces conneries jediscajedisrien']
['beaulieubap pir montre vrai visage fondamentalement extremedroite  homophobe sexiste']
['sexisme hopital atteint paroxysme bloc operatoire 75 hommes chez chirurgiennes ecarte cherie hahaha pourunemeuf sexisme payetablouse']
['mec qui a pris 1ere fois secret nai jamais dit personne balancetonporc']
['argent ambition pouvoir trois termes qui prennent tonalite negative quand on associe aux femmes il temps de deconstruire ces modeles pour briser plafond de verre via lesechos sexisme']
['propos misogynes de robinreda doivent etre sanctionnes misogynie fait partie violences infligees a

['personne forte celle qui peut fondre larmes pour ensuite reprendre armes continuer de ce battre inlassablement  resilience moiaussi jenelachejamaisrien']
['rmcinfo celinepiques ca met femme valeur faisant defiler bikini ensuite ca invite balancetonporc missfrance2018']
['sur gulli timber carte tresor particulierement sexiste insultant pour filles sexismepasnotregenre gulli']
['relancer viols hommes dans prisons syriennes plein balancetonporc discours unique autour de situations differentes cheu po']
['catherine deneuve  je suis femme libre demeurerai  est aux victimes elles elles seules presente excuses via libe']
['attention dans quelques minutes feminazies arreteront de faire pronostics sur concours de femmes recommenceront vous traquer coup de balancetonporc si vous leur faites compliment missfrance2018']
['pensees particulieres pour cyntoia brown zenash gezmu mariereine d victimes identifiables de violence patriarcale  communaute femmes women tobeawoman etreunefemme patriarcat vi

['lehuffpost jannonce nivellement salaires par bas par soucis egalite hein rassurez vous victimes de balancetonporc  gros bonnets auront toujours primes tous genres pour asseoir leur superiorite']
['bravo continuez critiquer aurore berge pour decollete jupe courte vous allez changer monde comme ca genies vous avez toujours pas compris quon na attendre votre validation pour habiller comme on veut  slt']
['amie une cousine qui divorce bout de 7 mois parce mari lui demande arreter etudes quil ne veut pas une femme beaucoup plus instruite lui']
['lemondefr  pourquoi aton besoin de preciser dylan farrow ronde  vous nauriez jamais dit un homme quil de embonpoint grossophobie sexisme']
['retrouvons methode internat de alliance francaise avec femme mari systematiquement abruti faible immature faisant toujours engueuler vous invite preter attention aux pubsbalancetonporc balancetatruie macron discrimination lemonde lefigaro']
['pere il a dit aujourhui truc 3600x lui fais oui papa sais merci il 

In [83]:
total_labels = len(y_ros_test)
acc = 0
for i in range(len(y_ros_test)):
    if(y_ros_test[i]==predictions[i]):
        acc = acc + 1
        
acc = acc / total_labels
print("Accuracy :" , acc)
        
metrics.confusion_matrix(y_ros_test, predictions)

Accuracy : 0.8954887218045112


array([[528, 137],
       [  2, 663]], dtype=int64)

### NAIVES BAYES + UNDERSAMPLING

In [64]:
data = pd.read_csv("my_csv_clean.csv",sep = ',') #we got that csv after running the Text Preprocessing file
data.columns = ['tweet', 'class']
    

X = data['tweet']
y = data['class'] 


categories = ["NotSexist", "Sexist"]

X_0 = []
X_1 = []
for i in range(len(data)):
    if data['class'][i]==0:
        X_0.append(data['tweet'][i])
        
for i in range(len(data)):
    if data['class'][i]==1:
        X_1.append(data['tweet'][i])

vec_0 = CountVectorizer()
X_c0 = vec_0.fit_transform(X_0)
tdm_0 = pd.DataFrame(X_c0.toarray(), columns=vec_0.get_feature_names())


vec_1 = CountVectorizer()
X_c1 = vec_1.fit_transform(X_1)
tdm_1 = pd.DataFrame(X_c1.toarray(), columns=vec_1.get_feature_names())

tdm_1

C:\Users\evara\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,02,04,06,10,100,1000,102,12,120,12000,...,yqb,yves,zaimerais,zamzonite,zenash,zeniss,zetais,zlatan,zohrabitan,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
word_list_0 = vec_0.get_feature_names();    
count_list_0 = X_c0.toarray().sum(axis=0) 
freq_0 = dict(zip(word_list_0,count_list_0))
freq_0

C:\Users\evara\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'000': 4,
 '10': 7,
 '100': 10,
 '1000': 1,
 '102': 2,
 '10h': 2,
 '10h00': 1,
 '10h30': 1,
 '11': 4,
 '1100': 1,
 '11000': 1,
 '117': 1,
 '12': 8,
 '1200': 1,
 '12000': 1,
 '120battementsparminute': 1,
 '1215': 1,
 '12h': 1,
 '12joursdaction': 1,
 '13': 7,
 '13h50': 1,
 '14': 9,
 '14eme': 1,
 '14h': 1,
 '14h00': 1,
 '14h15': 2,
 '15': 11,
 '157garlamd': 1,
 '15min': 1,
 '15x': 1,
 '16': 4,
 '169': 1,
 '16days': 2,
 '16h': 3,
 '16jours': 3,
 '17': 7,
 '1700': 1,
 '1703': 1,
 '1730': 1,
 '18': 9,
 '180': 1,
 '18006675005': 1,
 '18006686868': 1,
 '1825': 1,
 '18711955': 1,
 '18h': 2,
 '18h00': 1,
 '18h30': 1,
 '18h55': 1,
 '19': 6,
 '1900': 1,
 '1904': 1,
 '1905': 1,
 '1920': 2,
 '1927': 2,
 '1929': 2,
 '195': 2,
 '1954': 1,
 '1962': 1,
 '1965': 1,
 '1970': 1,
 '1979': 1,
 '1986': 1,
 '1987': 1,
 '1992': 1,
 '1999': 1,
 '19e': 1,
 '19h': 1,
 '19h30': 1,
 '19hruthelkrief': 2,
 '1a0maiscpasperdu': 1,
 '1dexmab': 1,
 '1dexmag': 1,
 '1e': 1,
 '1er': 10,
 '1ere': 17,
 '1iran': 1,
 '1les': 1,

In [66]:
word_list_1 = vec_1.get_feature_names();    
count_list_1 = X_c1.toarray().sum(axis=0) 
freq_1 = dict(zip(word_list_1,count_list_1))
freq_1

{'02': 3,
 '04': 3,
 '06': 1,
 '10': 6,
 '100': 2,
 '1000': 1,
 '102': 1,
 '12': 5,
 '120': 2,
 '12000': 1,
 '12f': 1,
 '13': 4,
 '14': 5,
 '14amp15': 1,
 '14ans': 1,
 '15': 2,
 '150eur': 1,
 '15ans': 1,
 '16': 2,
 '16ans': 1,
 '16days': 2,
 '16jours': 3,
 '17': 1,
 '1964': 1,
 '1990': 1,
 '1992': 2,
 '1997': 2,
 '1998': 1,
 '1999': 1,
 '1cm': 1,
 '1dexmag': 1,
 '1er': 1,
 '1ere': 3,
 '20': 7,
 '2004': 1,
 '2005': 1,
 '2006': 1,
 '2011': 1,
 '20122017': 1,
 '2014': 1,
 '2015': 2,
 '2016': 2,
 '2017': 9,
 '2018': 15,
 '20ans': 1,
 '20hfrance2': 1,
 '20minutes': 2,
 '21': 2,
 '21eme': 2,
 '22': 4,
 '23': 1,
 '24': 1,
 '2499': 1,
 '24hpujadas': 1,
 '25': 2,
 '25janvier': 2,
 '26': 1,
 '26caroll': 1,
 '27': 2,
 '28': 1,
 '2choses': 1,
 '2h': 1,
 '30': 4,
 '31': 1,
 '32': 1,
 '36': 1,
 '3600x': 1,
 '364': 1,
 '3784e': 1,
 '38': 2,
 '3eme': 1,
 '3min': 1,
 '40': 1,
 '40t': 1,
 '44': 1,
 '44t': 1,
 '4h': 1,
 '4meur': 1,
 '50': 5,
 '500': 1,
 '5000': 1,
 '50100500': 1,
 '50euros': 1,
 '50nuanc

In [67]:
prob_0 = []
for word,count in zip(word_list_0,count_list_0):
    prob_0.append(count/len(word_list_0))
dict(zip(word_list_0,prob_0))


{'000': 0.00043677658877484165,
 '10': 0.0007643590303559729,
 '100': 0.0010919414719371041,
 '1000': 0.00010919414719371041,
 '102': 0.00021838829438742082,
 '10h': 0.00021838829438742082,
 '10h00': 0.00010919414719371041,
 '10h30': 0.00010919414719371041,
 '11': 0.00043677658877484165,
 '1100': 0.00010919414719371041,
 '11000': 0.00010919414719371041,
 '117': 0.00010919414719371041,
 '12': 0.0008735531775496833,
 '1200': 0.00010919414719371041,
 '12000': 0.00010919414719371041,
 '120battementsparminute': 0.00010919414719371041,
 '1215': 0.00010919414719371041,
 '12h': 0.00010919414719371041,
 '12joursdaction': 0.00010919414719371041,
 '13': 0.0007643590303559729,
 '13h50': 0.00010919414719371041,
 '14': 0.0009827473247433938,
 '14eme': 0.00010919414719371041,
 '14h': 0.00010919414719371041,
 '14h00': 0.00010919414719371041,
 '14h15': 0.00021838829438742082,
 '15': 0.0012011356191308146,
 '157garlamd': 0.00010919414719371041,
 '15min': 0.00010919414719371041,
 '15x': 0.000109194147193

In [68]:
prob_1 = []
for word,count in zip(word_list_1,count_list_1):
    prob_1.append(count/len(word_list_1))
dict(zip(word_list_1,prob_1))


{'02': 0.0005337128624799857,
 '04': 0.0005337128624799857,
 '06': 0.0001779042874933286,
 '10': 0.0010674257249599715,
 '100': 0.0003558085749866572,
 '1000': 0.0001779042874933286,
 '102': 0.0001779042874933286,
 '12': 0.000889521437466643,
 '120': 0.0003558085749866572,
 '12000': 0.0001779042874933286,
 '12f': 0.0001779042874933286,
 '13': 0.0007116171499733144,
 '14': 0.000889521437466643,
 '14amp15': 0.0001779042874933286,
 '14ans': 0.0001779042874933286,
 '15': 0.0003558085749866572,
 '150eur': 0.0001779042874933286,
 '15ans': 0.0001779042874933286,
 '16': 0.0003558085749866572,
 '16ans': 0.0001779042874933286,
 '16days': 0.0003558085749866572,
 '16jours': 0.0005337128624799857,
 '17': 0.0001779042874933286,
 '1964': 0.0001779042874933286,
 '1990': 0.0001779042874933286,
 '1992': 0.0003558085749866572,
 '1997': 0.0003558085749866572,
 '1998': 0.0001779042874933286,
 '1999': 0.0001779042874933286,
 '1cm': 0.0001779042874933286,
 '1dexmag': 0.0001779042874933286,
 '1er': 0.00017790

In [69]:
X_train, X_test, y_train , y_test = train_test_split(X , y ,test_size=0.3)

# instantiating the random over sampler 
rus = RandomUnderSampler()
# resampling X, y
X_rus_train, y_rus_train = rus.fit_resample(np.array(X_train).reshape(-1,1), y_train)# new class distribution 
X_rus_test, y_rus_test = rus.fit_resample(np.array(X_test).reshape(-1,1), y_test)# new class distribution 

X_train_0 = []
X_train_1 = []

for i in range(len(X_rus_train)):
    if (y_ros_train[i]==0):
        X_train_0.append(X_rus_train[i].tolist()[0])
    else:
        X_train_1.append(X_rus_train[i].tolist()[0])

In [70]:
vec_0 = CountVectorizer()
X_vec_0 = vec_0.fit_transform(X_train_0)

total_features0 = len(vec_0.get_feature_names())
total_features0


C:\Users\evara\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


5450

In [71]:
vec_1 = CountVectorizer()
X_vec_1 = vec_1.fit_transform(X_train_1)

total_features1 = len(vec_1.get_feature_names())
total_features1

proba0 = total_features0 / (total_features0+total_features1)
proba1 = total_features1 / (total_features0+total_features1)

print(proba0)
print(proba1)

0.6165158371040724
0.3834841628959276


In [72]:
total_cnts_features_0 = count_list_0.sum(axis=0)
total_cnts_features_1 = count_list_1.sum(axis=0)

In [73]:
new_sentence = 'what is the price of the book'
new_word_list = word_tokenize(new_sentence)

In [74]:
predictions = []
for i in X_rus_test:
    print(i)
    proba_class0 = proba_sentence_class0(i[0])
    proba_class1 = proba_sentence_class1(i[0])
    if proba_class0 > proba_class1:
        predictions.append(0)
    else:
        predictions.append(1)
        
print(predictions)

['fil de presse sitesisyphetout ce trump fait pour ou contre femmes  mythe de virilite  pourquoi + de femmes technologies  25 janvier 1ere journee contre sexisme']
['si seulement militants de moiaussi qui adorent margaret atwood quand elle denonce derives totalitaires dans servante ecarlate ecoutaient aussi quand elle met garde contre leurs propres derives']
['travail domicile kate middleton enceinte remede contre nausees matinales devoile']
['quatre adolescentes poursuivent frappent jeune femme dans metro via 20minutes']
['la saoudienne devient femme comme autres elle aura aussi droit de faire traiter de salope volant moniakashmire ltd']
['liissiaa bah oui moi comprends jappellerais fille foufouna fils dar  ironie']
['yvelines neige verglas alerteorange du mercredi 23 fevrier 23h00 jeudi 1er mars 14h00 chutes de neige pluies verglacantes sont attendues soyez prudents suivez conseils de comportement ']
['ceux la on aime bien sia2018']
['quand balancetonporc fait derailler proces tron  

['cher ami  penelope fillon plus meme d eclairer votre lanterne ']
['entre lolobababa jeanfijanssens rien ne va plus  retrouvez ce soir 1905 dans slt sur c8tv ']
['conseils de segolene royal me manquent beaucoup pour lutter contre canicule2017']
['dueze blog mediasun animateur de radio belge pousse depart apres debat polemique sur viol']
['drole de defense de catherine deneuve par pierre arditi dans cavous']
['mais elle vraiment parfaite cette fille ironie']
['oh personnalites de annee selon time celles dont voix ont lance mouvement moiaussi metoo']
['apres segolene royal harlem desir voici marisol touraine  marche recase tous mauvais ministres socialistes dont on ne veut plus']
['sinon justice ou sur jugement deux pervers balancetonporc ']
['c sortie guignols psles francais ne veulent plus entendre parler de vous messieurs hollande lefoll hamon  il manque plus sapin nid de parasites e completah il manque segolene royal bah non elle trop peur de ouvrir pr pas faire virer']
['ce qui bie

['apres titre coup de poing de chillaofficiel balancetonporc freestyle poignant metoo de vinswaza enfonce clou']
['cliches sexistes dans pub  grandes marques engagent present dans programme faire lire article = sexisme']
['25 jan nee maud wood park 18711955 militante feministe suffragiste etatsunienne 1ere presidente de league of women votersest connue pour discours fougueux faveur du 19e amendement donnant dt de vote aux femmesqui finit par entrer vigueur 1920']
['merci aphp pour communication contre sexisme avec lettre de martinhirsch est long combat qui doit etre mene  bon debut ']
['sandramuller creatrice de balancetonporc poursuivie diffamation']
['hcefh membre du collectif ensemblecontrelesexisme reunissant 22 organisations associations qui organise journee nationale de debats echanges 250118 paris']
['britneyplage alabandine tout normal pour qui  toutestnormal trump usa whitepower populisme nationalisme fascisme libye racisme esclavage homophobie islamisme terrorisme fanatisme f

['sinon justice ou sur jugement deux pervers balancetonporc ']
['metoo  suede ebranlee par tempete lapresseplus moiaussi']
['parite ce debut annee 2018dans tous domaines femmes organisent  chu  femmes disent non sexisme reclament parite  via ladepechedumidi']
['petite fille qui joue faire sandwich avec de boue comme est mignon thewalkingdead ironie']
['artistes femmessolidaire vous disent merci laurythilleman bigup bientot on continue ensemblecontrelesexisme unefoispourtoutes']
['ce headshot de valerie pecresse contre laurent wauquiez ce matin on va pas en remettre ']
['mari de theresa may travaille pour gros actionnaire de bae systems lockheed martin fabricants missiles lances sur syrie tiens donc surprenant pas vraiment  malheur uns fait bonheur des']
['jour de match aujourhui pour nos bleues rennes 16h saint gregoire rennes metropole 9eme journee du championnat d2']
['frmarien jmdereeper emmanuelmacron bgriveaux auroreberge leclercerwan duchenecaro cyrielleaguedar dbaichere pierresa

['personne ne s occupe de l intelligence de ces miss sur twitter mais c bombe reste sur physique ca oui memes feront de bonne conscience demain avec  balancetonporc tout cela hilarant missfrance2018']
['ils veulent femme avec du caractere mais pas sauvage de mieux mieux hommes st pas complique du touuut ironie']
['rt payetashnek un sourire de toi quitte exfemme paris  solidays payetashnek']
['niveau de ouf  groscretin sexiste']
['main pose sur  fesse droite  une jeune femme pendant assemblee elle souffre anxiete de crises de panique de tendances suicidaires pendant 20ans fait du prix nobel de paix elie wiesel porc mais ou vaton  defendtonporc balancetonporc']
['moins epoque sexisme discrimination femmes faisait grand jour clairement dit de nos jours est meme chose mais plus sournoisement plus dans nondit ']
['ferme gueule connasse toujours memes qui ouvrent pour dire de merde']
['philousports matblaize putain vieux tir de femme enceinte ligue1 webgirondins']
['maitreeolas rescape pours

['putain cash quoi  payetashnek toi me fais saliver jte leche jusquau terminus lille  dans metro payetashnek']
['balancetonporc milo qui avait eu comportement inadapte sur moi amie']
['donald trump theresa may emmanuel macron  empereur femme petit prince nonalaguerre feteamacron']
['user de petit pouvoir pour faire sucer imagines mentalite de minable quil faut pour arriver  du petit chef quil faut etre  de laideur  du danger potentiel quil represente pour tous subordonnes  balancetonporc sauf il ministre']
['jetais heureuse ca parle de balance porc ce vois ca missfrance2018']
['maintenant est femme  bigre  ===gt gerald darmanin revele avoir ete accuse de viol par homme via valeurs']
['parispasrose lamanifpourtous komitidfr theresamay pas etonnant mere boutin presente anglais comme assassins enfants  on peut penser ce quon veut de theresa may mais moi dis respect ']
['dans quotidien on parle de didierroustan de expression  tir de femme enceinte normal pour tirer faut avoir ballon']
['en

['est quand meme belle femme segolene royal mais laurence ferrari mis trop de fond de teint tf1 journal']
['femme habillee une certaine maniere mais vierge pute gars qui fourre quequette partout c guerrier who raised yall']
['est bien comme couverture de foutre raciste homophobe sexiste  vous vous souvenez faire barrage fn etait il y 2 mois ']
['quand il y violences repetees faites aux femmes par homme politique hommes affaires ou star media ou du showbiz chaque fois beaucoup de gens savaient mais nont rien dit ou fait quand accusations de complicite laloisansfaiblesse balancetonporc']
['dire 14amp15 mai 2011 jetais formation feminisme amp socialisme ironie jai mal feminisme vous ']
['video honteuse scandaleuse degradante humiliante sexiste antigros sympa pour loana femme de castaldi']
['uniformisons car ttes situations sont identiques ironie mt ldeboissieu obliger homme prendre meme conge parental femme']
['me demandais si auroreberge portait soutif il me semble quon vient de voir sui

In [75]:
total_labels = len(y_rus_test)
acc = 0
for i in range(len(y_rus_test)):
    if(y_rus_test[i]==predictions[i]):
        acc = acc + 1
        
acc = acc / total_labels
print("Accuracy :" , acc)
        
metrics.confusion_matrix(y_rus_test, predictions)

Accuracy : 0.8982758620689655


array([[233,  57],
       [  2, 288]], dtype=int64)

In [76]:
# ==> ajouter les poids
# ==> meler plusieurs methodes
# ==> tester avec des paramètres différents 
# ==> faire un joli fichier